In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
#from langchain_community.document_loaders import PyPDFLoader
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings
import pandas as pd
from langchain.vectorstores import FAISS
import streamlit as st
from langchain.llms import CTransformers
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
import os

d:\Data-Science\Gen_AI\LLM\Langchain\Document-based-Q-and-A\docQnA\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [2]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [3]:
import pandas as pd

In [4]:
from langchain.vectorstores import FAISS

In [5]:
from InstructorEmbedding import INSTRUCTOR

In [6]:
import streamlit as st
from langchain.llms import CTransformers
from langchain.prompts import PromptTemplate

#embeddings = HuggingFaceInstructEmbeddings(model_name="WhereIsAI/UAE-Large-V1")
embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",model_kwargs={"device":"cpu"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [7]:
import os

In [8]:
folder_path=os.getcwd()

In [9]:
print(folder_path)

d:\Data-Science\Gen_AI\LLM\Langchain\Document-based-Q-and-A\docQnA


In [10]:
llm=CTransformers(
        #model='D:\Data-Science\Gen_AI\LLM\Langchain\email-generator\models\llama-2-7b-chat.ggmlv3.q8_0.bin',
        model='TheBloke/Llama-2-7B-Chat-GGUF',
        #model='syedzaidi-kiwi/Llama-2-7b-chat-finetune',
        model_type='llama',
        config={'max_new_tokens':256,
                'temperature':0}
    )

Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [11]:
def document_loader():
    loader=DirectoryLoader(f'{folder_path}\\Data',glob="./*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [12]:
def split_documents():
    documents=document_loader()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    texts=text_splitter.split_documents(documents)
    return texts

In [13]:
if os.path.exists(f"{folder_path}\\faiss_index_docQnA"):
    print("Vector Store exists")
    VectorStore=FAISS.load_local(f"{folder_path}\\faiss_index_docQnA",embeddings,allow_dangerous_deserialization=True)
else:
    print("Vector Store does not exists. Creating first time hence will take few minutes")
    os.makedirs(f"{folder_path}\\faiss_index_docQnA")
    print(f"The Vector store creation is in progress ....")
    texts=split_documents()
    VectorStore=FAISS.from_documents(texts,embeddings)
    print(f"The Vector store creation is done ....")
    VectorStore.save_local(f"{folder_path}\\faiss_index_docQnA")
    print(f"Vector DB is saving to local drive at {folder_path}\\faiss_index_docQnA Finished!!")

Vector Store exists


In [14]:
VectorStore

In [15]:
retriver=VectorStore.as_retriever(search_kwargs={"k":3})

In [16]:
retriver.get_relevant_documents("What is the experience summary of Ishu?")

[Document(page_content='Ishu Srivastava  \nPMP Certified Professional  \n \nDynamic and results -driv en professional with a focus on technical project \nmanagement and data science. Seeking opportunities in Noida or Delhi/NCR \nregion to leverage expertise in project management practices and hands -on \ntechnical skills.  \n  \n \nE-MAIL   ishu.cs@gmail.com                                      \nMOBILE    +91 -9953697986  \nLINKEDIN: www.linkedin.com/in/ishu -\nsrivastava/  \nGITHUB: https://github.com/ishu -cs', metadata={'source': 'D:\\Data-Science\\Gen_AI\\LLM\\Langchain\\Document-based-Q-and-A\\docQnA\\Data\\Ishu_Srivastava.pdf', 'page': 0}),
 Document(page_content='➢ Played a pivotal role in ensuring regulatory compliance and data integrity by implementing rigorous data governance policies \nand procedures.  \n➢ Collaborated closely with stakeholders to define and prioritize database requirements, aligning technical solutions with busi ness \nobjectives and goals.  \n \n PREVIOUS

In [17]:
def generate_qa_chain(VectorStore):
    retriver=VectorStore.as_retriever(search_kwargs={"k":3})
    qa_chain=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=retriver.get_relevant_documents,input_key="question")
    return qa_chain,retriver

In [25]:
def get_user_input_and_execute_function():
    # Get input from the user
    user_input = input("Please enter something (type 'stop' to quit): ")
    
    # Check if user wants to stop
    if user_input.lower() == 'stop':
        print("Stopping program...")
        return False
    
    # Call the function with the user input as parameter
    my_function(user_input)
    
    return True

In [26]:
questions = []
answers = []
ref_doc = []
page =[]

In [27]:
def load_embeddings(path,embeddings):
  VectorStore=FAISS.load_local(f"{path}\\faiss_index_docQnA",embeddings,allow_dangerous_deserialization=True)
  return VectorStore

In [28]:
db_instructEmbedd = load_embeddings(path=f'{folder_path}',embeddings=embeddings)#,allow_dangerous_deserialization=True)

In [29]:
retriever = db_instructEmbedd.as_retriever(search_kwargs={"k":1})

In [30]:
qa_chain_instrucEmbed = RetrievalQA.from_chain_type(llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="question"
    #return_source_document = True
)

In [20]:
qa_chain=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",input_key="question",retriever=retriver.get_relevant_documents("question"))

ValidationError: 1 validation error for RetrievalQA
retriever
  value is not a valid dict (type=type_error.dict)

In [31]:
answer = qa_chain_instrucEmbed("What is the experience summary of Ishu?")
# print(answer)

d:\Data-Science\Gen_AI\LLM\Langchain\Document-based-Q-and-A\docQnA\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [43]:
print(answer)

{'question': 'What is the experience summary of Ishu?', 'result': ' Based on the provided information, I can see that Ishu has 10 years of experience in project management and data science. He is PMP certified and has worked in various industries including IT, Finance, and Manufacturing. He has also worked as a freelancer and has experience in leading teams and managing projects remotely.\n\nPlease answer the question based on the provided information.'}


In [20]:
def my_function(user_input):
    questions.append(user_input)
    ans = qa_chain_instrucEmbed(user_input)
    answers.append(ans.get('result'))
    ref_doc.append(retriever.get_relevant_documents(user_input)[0].metadata.get('source'))
    page.append(retriever.get_relevant_documents(user_input)[0].metadata.get('page'))